# Example ➊➋➌ <-> ➋➌➊ <-> ➌➊➋ <-> ➊➋➌

*Objective:* Demonstrate state transfer where substrates and products overlap.

The model has a three-carbon compound and a reversible reaction that moves the first atom to the last position. At equilibrium, all labeling states should exist at the same concentration.

Reactions:
    ➊➋➌ <-> ➋➌➊ <-> ➌➊➋ <-> ➊➋➌

## Model definition

In [ ]:
from isrene.model_definition import *
from isrene import ureg, Q_
from isrene.model_definition.model_export_sbml import ModelExporterSbmlMulti
from isrene.sbml_multi.sbml_multi_expand import NetworkGenerator

import numpy as np

In [ ]:
# Create the model
model = Model("cycle")

compartment = Compartment("C", ureg.Quantity(1, ureg.mL), 3, model=model)

S = Metabolite("S", num_labelable_carbons=3, model=model)

AtomMappingReaction(
    "abc_bca",
    [(S(compartment=compartment), "abc")],
    [(S(compartment=compartment), "bca")],
    reversible=True,
    rate_law_generator=MassActionRateLawGenerator(),
)

InitialConcentration(
    S({"C1": "l", "C2": "u", "C3": "u"}, compartment=compartment),
    Q_(1.0, ureg.mM),
);

## Write model to SBML multi and expand to SBML core

In [ ]:
# write SBML multi
multi_file = f"tmp/{model.id}_multi.xml"
exporter = ModelExporterSbmlMulti(model)
exporter.export(multi_file)

from IPython.display import display, Markdown
from pathlib import Path
# from IPython.display import Code

# Code(Path(multi_file).read_text(), language='xml')

display(Markdown(f"```xml\n{Path(multi_file).read_text()}\n```"))

In [ ]:
# expand network and save SBML core model
core_model_name = f"tmp/{model.id}_core.xml"
nwg = NetworkGenerator(model=multi_file)
nwg.create_sbml_core_model(core_model_name)

In [ ]:
import libsbml
sbml_doc = libsbml.SBMLReader().readSBMLFromFile(core_model_name)
sbml_model = sbml_doc.getModel()
print("Species in core model:", [s.getId() for s in sbml_model.getListOfSpecies()])

## Import to AMICI

In [ ]:
from amici.sbml_import import SbmlImporter

amici_model_name = model.id
amici_model_dir = f"tmp/{amici_model_name}"

SbmlImporter(core_model_name).sbml2amici(
    model_name=amici_model_name, output_dir=amici_model_dir, verbose=False
)

## Simulate model

In [ ]:
import amici

model_module = amici.import_model_module(
    amici_model_name, module_path=amici_model_dir
)
amici_model = model_module.getModel()

print(amici_model.getParameterIds())

In [ ]:
# Set model parameters
amici_model.setParametersByIdRegex(".*", 1.0)
amici_model.setParameterById("kf_abc_bca", 5e-3)

amici_model.setTimepoints(list(range(1000)))
solver = amici_model.getSolver()
rdata = amici.runAmiciSimulation(amici_model, solver)

from amici.plotting import plot_state_trajectories

plot_state_trajectories(rdata, model=amici_model)

In [ ]:
# total amount is conserved
assert np.isclose(
    1,
    rdata.by_id("S_C_C1_l_C2_u_C3_u")
    + rdata.by_id("S_C_C1_u_C2_l_C3_u")
    + rdata.by_id("S_C_C1_u_C2_u_C3_l"),
).all()

In [ ]:
# at steadystate, all concentrations are the same
assert np.isclose(
    rdata.by_id("S_C_C1_l_C2_u_C3_u")[-1],
    rdata.by_id("S_C_C1_u_C2_l_C3_u")[-1],
    rtol=1e-6,
)
assert np.isclose(
    rdata.by_id("S_C_C1_l_C2_u_C3_u")[-1],
    rdata.by_id("S_C_C1_u_C2_u_C3_l")[-1],
    rtol=1e-6,
)